# Synchronous Generator VBR model - Operational against Fundamental Params Validation

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import subprocess

# %matplotlib widget
timestep = 10e-6
duration = 1.0

begin_idx = 0
end_idx = -1

dpsim_path = (
    subprocess.Popen(["git", "rev-parse", "--show-toplevel"], stdout=subprocess.PIPE)
    .communicate()[0]
    .rstrip()
    .decode("utf-8")
)
path_exec = dpsim_path + "/build/dpsim/examples/cxx/"

## Fundamental parameters

In [ ]:
model_name_fundamental = "EMT_SynGenVBR_SMIB_Fault"

sim = subprocess.Popen(
    [
        path_exec + model_name_fundamental,
        "--name",
        model_name_fundamental,
        "--timestep",
        str(timestep),
        "--duration",
        str(duration),
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)
print(sim.communicate()[0].decode())

path_res = "logs/" + model_name_fundamental + "/"
dpsim_result_file_res = path_res + model_name_fundamental + ".csv"
ts_fundamental = read_timeseries_csv(dpsim_result_file_res)

## Operational parameters

In [ ]:
model_name_operational = "EMT_SynGenVBR_OperationalParams_SMIB_Fault"

sim = subprocess.Popen(
    [
        path_exec + model_name_operational,
        "--name",
        model_name_operational,
        "--timestep",
        str(timestep),
        "--duration",
        str(duration),
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)
print(sim.communicate()[0].decode())

path_res = "logs/" + model_name_operational + "/"
dpsim_result_file_res = path_res + model_name_operational + ".csv"
ts_operational = read_timeseries_csv(dpsim_result_file_res)

## Genrerator terminal Current

In [ ]:
plt.figure(figsize=(12, 8))
plt.ylabel("Generator terminal current (V)", fontsize=18)

for name in ["i_gen"]:
    plt.plot(
        ts_fundamental[name + "_0"].time[begin_idx:end_idx],
        np.sqrt(3 / 2) * ts_fundamental[name + "_0"].values[begin_idx:end_idx],
        label=name,
    )
    plt.plot(
        ts_operational[name + "_0"].time[begin_idx:end_idx],
        np.sqrt(3 / 2) * ts_operational[name + "_0"].values[begin_idx:end_idx],
        linestyle="--",
        label=name,
    )

plt.legend()
plt.show()

In [ ]:
assert (
    np.max(
        ts_fundamental[name + "_0"].values[begin_idx:end_idx]
        - ts_operational[name + "_0"].values[begin_idx:end_idx]
    )
    / np.max(ts_fundamental[name + "_0"].values[begin_idx:end_idx])
    < 1e-3
)